# Red Neuronal Profunda (DNN) para clasificación MNIST

Aplicaremos todos nuestros conocimientos para crear una DNN.  El problema que vamos a trabajar se conoce como el "Hola Mundo" del aprendizaje profundo porque para la mayoría de estudiantes este es el primer algoritmo de aprendizaje profundo que ven. 

El conjunto de datos se llama MNIST y se refiere al reconocimiento de dígitos escritos a mano.  Pueden encontrar más información en el sitio web de Yann LeCun (Director of AI Research, Facebook).  El es uno de los pioneros de todo este tema, así como de otras metodologías más complejas como las Redes Neurales Convolucionales (CNN) que se utilizan hoy día.

El conjunto de datos tiene 70,000 imágenes (28x28 pixels) de dígitos escritos a mano (1 dígito por imagen).

La meta es escribir un algoritmo que detecta qué dígito ha sido escrito.  Como solo hay 10 dígitos (0 al 9), este es un problema de clasificación con 10 clases.

Nuestra meta será construir una RN con 2 capas escondidas.

## Importar los paquetes relevantes

TensorFlow incluye un proveedor de datos de MNIST que utilizaremos acá.  Viene con el módulo **"tensorflow-datasets"** por lo que si no lo ha instalado aún, debe hacerlo:

pip install tensorflow-datasets

ó

conda install tensorflow-datasets

Estos conjuntos de datos se almacenarán en su directorio C:\Users\usuario\tensorflow_datasets|...

La primera vez que baje un conjunto de datos, se almacenará en la carpeta respectiva.  Cada vez subsiguiente, automáticamente cargará la copia en su computadora

In [116]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Datos

Esta sección es donde pre-procesaremos nuestros datos.

**tfd.load** carga un conjunto de datos (o si es la primera vez, los baja y luego los carga).  En este caso estamos interesados en el conjunto de datos MNIST.  El único argumento obligatorio es el nombre del conjunto de datos.  Hay otros que pueden ser útiles, por ejemplo:

**with_info = True** nos provee con una tupla que contiene información sobre la versión, features, número de observaciones (samples)

**as_supervised = True** cargará el conjunto de datos en una estructura de 2 tuplas (entrada, meta).  Si se usa **False**, retorna un diccionario, obviamente preferimos tener de una vez nuestra entrada y meta separados.

In [117]:
datos_mnist, info_mnist = tfds.load(name='mnist',
                                    shuffle_files = False,
                                    with_info=True, 
                                    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete886Q9L\mnist-train.tfrecord*...:   0%|          | 0/6000…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\mnist\3.0.1.incomplete886Q9L\mnist-test.tfrecord*...:   0%|          | 0/10000…

Dataset mnist downloaded and prepared to ~\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


Una vez se ha cargado el conjunto de datos, se pueden, fácilmente, extraer los conjuntos de entrenamiento y prueba.

In [118]:
entreno_mnist, prueba_mnist = datos_mnist['train'], datos_mnist['test']

Por default, TF2 tiene conjuntos de datos de entrenamiento y de prueba, pero no tiene un conjunto de validación, por lo que debemos dividirlo por nuestra cuenta

Empezamos por definir el número de observaciones de validación, como un porcentaje de las observaciones de entrenamiento.  Aqui es donde también usamos **mnist_info** (no tenemos que contar las observaciones)

In [119]:
num_obs_validacion = 0.1 * info_mnist.splits['train'].num_examples

Convertimos este número a entero ya que un float puede causar problemas en el camino

In [120]:
num_obs_validacion = tf.cast(num_obs_validacion, tf.int64)

Usaremos una variable dedicada para el número de muestras de prueba

In [121]:
num_obs_prueba = info_mnist.splits['test'].num_examples

In [122]:
num_obs_prueba = tf.cast(num_obs_prueba, tf.int64)

Normalmente preferimos "normalizar" nuestros datos en alguna forma para que el resultado sea numéricamente más estable.  En este caso simplemente preferimos tener entradas entre 0 y 1, por lo que definimos una función, que reciba la imagen MNIST y su etiqueta, para hacerlo.

Como los posibles valores de las entradas son entre 0 y 255 (256 posibles tonos de gris), al dividirlos por 255 obtenemos el resultado deseado.

In [123]:
def normalizar(imagen, etiqueta):
    imagen = tf.cast(imagen, tf.float32)
    imagen /= 255.
    return imagen, etiqueta

El método .map() nos permite aplicar una transormación "customizada" a un conjunto de datos.  Ya hemos decidido que obtendremos los datos de validación a partir de *mnist_train*

In [124]:
datos_entrenamiento_y_validacion_normalizados = entreno_mnist.map(normalizar)

Finalmente, normalizaremos y convertiremos los datos de pruebas en tandas.  Los normalizamos para que tengan la misma magnitud que los datos de entrenamiento y validación.

No hay necesidad de "barajearlo" ya que no estaremos entrenando con los datos de prueba.  Habra una sola tanda, igual al tamaño de los datos de prueba.

In [125]:
datos_prueba = prueba_mnist.map(normalizar)

Si "barajearemos" los datos de entrenamiento y validación.

El parámetro **TAMANIO_BUFFER** se utiliza para casos que tengan conjuntos de datos grandes.  En este caso no es posible "barajear" el conjunto completo de un solo porque no cabe en la memoria.  En vez, TF2 solo almacena los datos en memoria **TAMANIO_BUFFER** muestras a la vez, y los "barajea".

si TAMANIO_BUFFER = 1 => no hay "barajeo"
si TAMANIO_BUFFER >= número de muestras => el "barajeo" se hace uniformemente

para un TAMANIO_BUFFER intermedio - se hace una optimización computacional para aproximar un "barajeo" uniforme.

Afortunadamente, hay un método de "barajeo" disponible y solo necesitamos especificar el tamaño del buffer.

In [126]:
TAMANIO_BUFFER = 10000

In [127]:
datos_entrenamiento_y_validacion_barajeados = datos_entrenamiento_y_validacion_normalizados.shuffle(TAMANIO_BUFFER)

Una vez se han "normalizado" y "barajeado" los datos, podemos proceder a extraer los datos de entrenamiento y de validación.

Nuestros datos de validación serán el 10% del conjunto de entrenamiento, que ya se calculó utilizando el método **.take()**.

Finalmente, creamos una tanda con un tamaño de tanda igual al total de muestras de validación.

In [128]:
datos_validacion = datos_entrenamiento_y_validacion_barajeados.take(num_obs_validacion)

Similarmente, los datos de entrenamiento son todos los demás por lo que nos salteamos tantas observaciones como las hay en el conjunto de validación.

In [129]:
datos_entreno = datos_entrenamiento_y_validacion_barajeados.skip(num_obs_validacion)

Establecemos el tamaño de las tandas.

También podemos aprovechar el momento para separar los datos de entrenamiento y de prueba.

Estos serán muy útiles cuando entrenemos, ya que podemos iterar sobre las diferentes tandas

In [130]:
TAMANIO_TANDA = 100

datos_entreno = datos_entreno.batch(TAMANIO_TANDA)

datos_validacion = datos_validacion.batch(num_obs_validacion)

datos_prueba = datos_prueba.batch(num_obs_prueba)

Toma la siguiente tanda (es la única tanda) ya que, como configuramos **as_supervized = True**, obtuvimos una estructura de 2 tuplas 

In [131]:
entradas_validacion, metas_validacion = next(iter(datos_validacion))

## Modelo

### Delineamos el modelo

Cuando pensamos sobre un algoritmo de aprenzaje profundo, casi siempre solo lo imaginamos.  Asi que esta vez, hagámoslo.  :)

In [132]:
tamanio_entrada = 784
tamanio_salida = 10

Usaremos el mismo ancho para ambas capas escondidas.  No es una necesidad!

In [133]:
tamanio_capa_escondida = 50

# Definimos cómo se verá el modelo

La primera capa (la de entrada):  cada observación es de 28x28x1 píxeles, por lo tanto es un tensor de rango 3.

Como aún no hemos aprendido sobre CNNs, no sabemos como alimentar este tipo de entrada a nuestra red, por lo tanto hay que "aplanar" las imágenes.  Hay un método conveniente **Flatten** que toma nuestro tensor de 28x28x1 y lo covierte en  un vector (None), o (784,)...porque 28x28x1 = 784.  Esto nos permite crear una red de alimentación hacia adelante.

    
**tf.keras.layers.Dense** básicamente implementa:  output = activation(dot(entrada, peso) + sesgo).  Requiere varios argumentos, pero los más importantes para nosotros son el ancho de la capa escondida y la función de activación.

La capa final no es diferente, solo nos aseguramos de activarla con **softmax**


In [134]:
modelo = tf.keras.Sequential([

    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # capa entrada
    
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 1era capa escondida
    tf.keras.layers.Dense(tamanio_capa_escondida, activation='sigmoid'), # 2nda capa escondida

    tf.keras.layers.Dense(tamanio_salida, activation='softmax') # capa salida
])

### Seleccionar el optimizador y la función de pérdida

Definimos el optimizador que nos gustaría utilizar, la función de pérdida, y las métricas que nos interesa obtener en cada interacción

In [135]:
modelo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Entrenamiento

Acá es donde entrenamos el modelo que hemos construído

Determinamos el número máximo de épocas.

Ajustamos el modelo , especificando:

* los datos de entrenamiento
* el número total de épocas
* y los datos de validación que creamos en el formato (entradas, metas)

In [136]:
NUMERO_EPOCAS = 5

modelo.fit(datos_entreno, 
          epochs = NUMERO_EPOCAS, 
          validation_data = (entradas_validacion, metas_validacion),
          validation_steps = 10,
          verbose = 2)

Epoch 1/5
540/540 - 2s - loss: 1.0071 - accuracy: 0.7650 - val_loss: 0.4516 - val_accuracy: 0.8875 - 2s/epoch - 3ms/step
Epoch 2/5
540/540 - 1s - loss: 0.3443 - accuracy: 0.9068 - val_loss: 0.2830 - val_accuracy: 0.9205 - 1s/epoch - 2ms/step
Epoch 3/5
540/540 - 1s - loss: 0.2500 - accuracy: 0.9294 - val_loss: 0.2231 - val_accuracy: 0.9377 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.2043 - accuracy: 0.9418 - val_loss: 0.1868 - val_accuracy: 0.9487 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.1729 - accuracy: 0.9500 - val_loss: 0.1610 - val_accuracy: 0.9543 - 1s/epoch - 2ms/step


## Probar el modelo

Como se discutió en clase, luego del entrenamiento (con los datos de entrenamiento), y la validación (con los datos de validación), probamos el potencial de predicción final de nuestro modelo con el conjunto de datos de prueba que el algoritmo NUNCA ha visto antes.

Es muy importante reconocer que estar "jugando" con los hiperparámetros sobre-ajusta el conjunto de datos de validación.

La prueba es la instancia absolutamente final. Nunca debe probarse el modelo antes de haber completamente ajustado el modelo.

Si se ajusta el modelo después de hacer la prueba, se empezará a sobre-ajustar el conjunto de datos de prueba, que echaría "por los suelos" el propósito original del mismo.

In [137]:
perdida_prueba, precision_prueba = modelo.evaluate(datos_prueba)

1/1 [==============================] - 0s 221ms/step - loss: 0.1676 - accuracy: 0.9495


In [138]:
# Si se desea, se puede aplicar un formateo "bonito"
print('Pérdida de prueba: {0:.2f}. Precisión de prueba: {1:.2f}%'.format(perdida_prueba, precision_prueba * 100.))

Pérdida de prueba: 0.17. Precisión de prueba: 94.95%


Utilizando el modelo inicial y los hiperparámetros dados en este notebook, la precisión de prueba final debe ser aproximadamente 97%.

Cada vez que se ejecuta el código, se obtiene una precisión diferente debido a la "barajeada" de las tandas, los pesos se inicializan en forma diferente, etc.

Finalmente, intencionalmente se ha llegado a una solución subóptima, para que pueda tener la oportunidad de mejorarla.